In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import socketio
import asyncio

from flask import Flask
from flask_socketio import SocketIO, send, emit

In [2]:
sio = socketio.Client()

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

In [3]:
@socketio.on('data', namespace="/recv_data")
async def got_data(data):
    print("Received data in part1")
    inpt = tf.convert_to_tensor(data)
    out = model(inpt).numpy().tolist()
    sio.emit('data', data=out, namespace="/recv_data")

@socketio.on('connect', namespace="/recv_model")
def connect(auth):
    print("Dispatcher Connected, auth:", auth["name"])

@socketio.on('connect', namespace="/recv_data")
def connect():
    print("Previous node connected")

@socketio.on('dispatch', namespace="/recv_model")
def recv_model(model_json, next_node):
    global model
    print(model_json)
    model = tf.keras.models.model_from_json(model_json)
    #model.save("models/part1")
    sio.connect(next_node, namespaces=['/recv_data'])

In [4]:
if __name__ == '__main__':
    socketio.run(app, port=5001)

WebSocket transport not available. Install simple-websocket for improved performance.


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
The WebSocket transport is not available, you must install a WebSocket server that is compatible with your async mode to enable it. See the documentation for details. (further occurrences of this error will be logged with level INFO)
127.0.0.1 - - [15/Sep/2021 17:07:06] "GET /socket.io/?transport=polling&EIO=4&t=1631750824.3091724 HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2021 17:07:08] "POST /socket.io/?transport=polling&EIO=4&sid=vxlLH42mvRI0wbB5AAAA HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2021 17:07:08] "GET /socket.io/?transport=polling&EIO=4&sid=vxlLH42mvRI0wbB5AAAA&t=1631750826.4095175 HTTP/1.1" 200 -


Dispatcher Connected, auth: dispatcher


127.0.0.1 - - [15/Sep/2021 17:07:10] "POST /socket.io/?transport=polling&EIO=4&sid=vxlLH42mvRI0wbB5AAAA HTTP/1.1" 200 -


{"class_name": "Functional", "config": {"name": "model", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [3, 2], "dtype": "int32", "sparse": false, "ragged": false, "name": "input_1"}, "name": "input_1", "inbound_nodes": []}, {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "dtype": "float32", "units": 3, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "Constant", "config": {"value": 1.0}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "name": "dense", "inbound_nodes": [[["input_1", 0, 0, {}]]]}], "input_layers": [["input_1", 0, 0]], "output_layers": [["dense", 0, 0]]}, "keras_version": "2.6.0", "backend": "tensorflow"}


Exception in thread Thread-13:
Traceback (most recent call last):
  File "C:\Users\arjun\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\arjun\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\arjun\AppData\Local\Programs\Python\Python39\lib\site-packages\socketio\server.py", line 712, in _handle_event_internal
    r = server._trigger_event(data[0], namespace, sid, *data[1:])
  File "C:\Users\arjun\AppData\Local\Programs\Python\Python39\lib\site-packages\socketio\server.py", line 736, in _trigger_event
    return self.handlers[namespace][event](*args)
  File "C:\Users\arjun\AppData\Local\Programs\Python\Python39\lib\site-packages\flask_socketio\__init__.py", line 282, in _handler
    return self._handle_event(handler, message, namespace, sid,
  File "C:\Users\arjun\AppData\Local\Programs\Python\Python39\lib\site-packages\flask_s

Previous node connected


127.0.0.1 - - [15/Sep/2021 17:07:21] "POST /socket.io/?transport=polling&EIO=4&sid=doNkpG7LwMgEjRskAAAC HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2021 17:07:31] "GET /socket.io/?transport=polling&EIO=4&sid=vxlLH42mvRI0wbB5AAAA&t=1631750828.4795856 HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2021 17:07:33] "POST /socket.io/?transport=polling&EIO=4&sid=doNkpG7LwMgEjRskAAAC HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2021 17:07:38] "POST /socket.io/?transport=polling&EIO=4&sid=doNkpG7LwMgEjRskAAAC HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2021 17:07:42] "GET /socket.io/?transport=polling&EIO=4&sid=doNkpG7LwMgEjRskAAAC&t=1631750839.8059137 HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2021 17:07:44] "POST /socket.io/?transport=polling&EIO=4&sid=doNkpG7LwMgEjRskAAAC HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2021 17:07:58] "GET /socket.io/?transport=polling&EIO=4&sid=doNkpG7LwMgEjRskAAAC&t=1631750851.4103754 HTTP/1.1" 200 -
C:\Users\arjun\AppData\Local\Programs\Python\Python39\lib\threading.py:542: RuntimeWarning: coroutin